# ARCH and BIO Audits

## outputs:
### ARC:
* ARCH check
* ARCH APN count central
### BIO:
* BIO checks
* Bio APN Counts central

## Downloads needed:
* 2 Smartsheets both Central and South or BOP and ANVIL
### ARC:
* 1 - Power BI download
### BIO:
* 1 - Power BI download

# Libraries

In [1]:
import pandas as pd
import glob
import numpy as np
import os
import smartsheet
import datetime

# Variables

In [2]:
csv = '.csv'
excel = '.xlsx'

In [3]:
while True:
    user = input('What project are we Auditing?: Central or Northern ').lower()
    try:

        #central Set up
        if user == 'central':
            print('Central Audit Started')
            # Counties:
            county= ['ALPINE', 'EL DORADO', 'LAKE', 'NEVADA', 'PLACER', 'TUOLUMNE', 'MENDOCINO', 'TULARE', 'KERN']

            # Division. * northern goes here becasue we want everything not in northern Division
            division = 'Northern Division'

            # smartsheets Shet IDs
            sheet_id1 = '1844440261257092' # central
            sheet_id2= '6292262490531716' # south central

            # Central's API
            API1 = smartsheet.Smartsheet('6jJFxnAfFsDErU0BSr1Ae4D6cHq0zenczVR12') # API central

            API1.errors_as_exceptions(True)

            # Central's API
            API2 = smartsheet.Smartsheet("H8mqAo60J2zGHSrJEwmlMoY88A9yjgCo6RnSI") # API south central
            API2.errors_as_exceptions(True)
            break

        #northern Set up
        elif user == 'northern':
            print('Northern Audit Started')
            # Counties:
            county = ['LASSEN', 'PLUMAS', 'SISKIYOU', 'TRINITY', 'SHASTA', 'TEHAMA']

            # Division. * northern goes here becasue we want everything not in northern Division
            division = 'Central Division'

            # smartsheets Shet IDs
            sheet_id1 = '3589680131270532' # BOP
            sheet_id2= '5367491607193476' # ANVIL

            # Central's API
            API1 = smartsheet.Smartsheet('6w7vusWNdvdm9TJmjImZfbh0SPQgi3HJaekNb') # API BOP

            API1.errors_as_exceptions(True)

            # Central's API
            API2 = smartsheet.Smartsheet("MQQmCjvvFWLKBoX9K7A33rHSoLnuj1MGVJuVK") # API ANVIL
            API2.errors_as_exceptions(True)
            break
    except ValueError:
        print("\ninvaild input Only 'Central or Northern' allowed")

What project are we Auditing?: Central or Northern northern
Northern Audit Started


# Function to set up Smartsheet into a readable Database

In [4]:
def smartsheet_loadup(sheet):
    columns = [col.title for col in sheet.columns]
    rows = []
    for row in sheet.rows:
        cells = []
        for cell in row.cells:
            cells.append(cell.value)
        rows.append(cells)
    df = pd.DataFrame(rows, columns=columns)
    return df

Area1_SS = smartsheet_loadup(API1.Sheets.get_sheet(sheet_id1))
Area2_SS = smartsheet_loadup(API2.Sheets.get_sheet(sheet_id2))

df_both = pd.concat([Area1_SS,Area2_SS])

# Read in ARCH adn BIO Power BI databases

In [5]:
path2 = r"C:\Users\jacque.trahan\Downloads\BIO*"
file_type2 = ".csv"
load_it2 = glob.glob(path2 + file_type2)

df2 = pd.read_csv(max(load_it2, key=os.path.getctime))

In [6]:
path3 = r"C:\Users\jacque.trahan\Downloads\Arch*"
file_type3 = ".csv"
load_it3 = glob.glob(path3 + file_type2)

df3 = pd.read_csv(max(load_it3, key=os.path.getctime))

In [7]:
smartsheets = df_both.copy()
bio = df2.copy()
arch = df3.copy()

# smart sheet Cleaning:
APN_ROW Segment 'Arch Monitor Needed','Bio Monitor Needed'

In [8]:
# select columns we need
smartsheets = smartsheets[['APN_ROW Segment','Street #','Street Name', 'Structural Status','County',
                           'Arch Monitor Needed','Arch Assessment',
                           'Bio Monitor Needed','Bio Assessment']]

In [9]:
smartsheets['Arch Assessment'] = pd.to_datetime(smartsheets['Arch Assessment']).dt.date
smartsheets['Bio Assessment'] = pd.to_datetime(smartsheets['Bio Assessment']).dt.date

In [10]:
smartsheets.rename(columns={'Arch Monitor Needed': 'Arch Monitor Needed_SS','Arch Assessment':'Arch Assessment_SS',
                            'Bio Monitor Needed':'Bio Monitor Needed_SS','Bio Assessment':'Bio Assessment_SS'}, inplace=True)

# BIO columns needed and cleaning:
APN , Biological monitoring recommended?

In [11]:
# select columns we need
bio = bio[['APN', 'Biological monitoring recommended?','County', 'Observation Date']]

In [12]:
# get just for central dixie fire
bio = bio[bio['County'].isin(county)]

### Get BIO APN count in Central

In [13]:
bio[['APN']].value_counts().to_excel('Bio APN Counts central.xlsx')

In [14]:
# bio['Biological monitoring recommended?'].value_counts()

In [15]:
bio['Observation Date'] = pd.to_datetime(bio['Observation Date']).dt.date

In [16]:
# change column name values to match smart sheets
bio['Biological monitoring recommended?'] = bio['Biological monitoring recommended?'].str.replace('No Debris, No Tree', 'No Bio Monitoring Needed for Debris or Trees')
bio['Biological monitoring recommended?'] =bio['Biological monitoring recommended?'].str.replace('Yes Debris, Yes Tree', 'Bio Monitoring Needed for Debris AND Trees')
bio['Biological monitoring recommended?'] =bio['Biological monitoring recommended?'].str.replace('No Debris, Yes Tree', 'Bio Monitoring Needed for Trees, not Debris')
bio['Biological monitoring recommended?'] =bio['Biological monitoring recommended?'].str.replace('Yes Debris, No Tree','Bio Monitoring Needed for Debris, not Trees')

In [17]:
bio.rename(columns={'Observation Date':'Observation Date_BIO'}, inplace=True)

# ARCH columns needed and cleaning:
APN , Archaeological Resources Present?

In [18]:
# select columns we need
arch = arch[['APN', 'Monitoring Recommendation','Date', 'County']]

In [19]:
# get just for central dixie fire
arch = arch[arch['County'].isin(county)]

### Get ARCH APN count for central

In [20]:
arch['APN'].value_counts().to_excel('ARCH APN count central.xlsx')

In [21]:
# formate dates 
arch['Date'] = pd.to_datetime(arch['Date']).dt.date

In [22]:
# change column name values to match smart sheets
arch['Monitoring Recommendation'] = arch['Monitoring Recommendation'].str.replace('No Debris, No Tree', 'No Arch Monitoring Needed for Debris or Trees')
arch['Monitoring Recommendation'] = arch['Monitoring Recommendation'].str.replace('Yes Debris, Yes Tree', 'Arch Monitoring Needed for Debris AND Trees')
arch['Monitoring Recommendation'] = arch['Monitoring Recommendation'].str.replace('No Debris, Yes Tree', 'Arch Monitoring Needed for Trees, not Debris')
arch['Monitoring Recommendation'] = arch['Monitoring Recommendation'].str.replace('Yes Debris, No Tree', 'Arch Monitoring Needed for Debris, not Trees')

In [23]:
arch.rename(columns={'Monitoring Recommendation':'Monitoring Recommendation_ARCH', 'Date': 'Date_ARCH'},inplace=True)

# Merge Data bases
* smartsheets and BIO
* smasrtsheet and Arch

# Smartsheet and BIO merge and additonal cleaning

In [24]:
# Merge Smartsheets and BIO

smartsheets_bio = smartsheets.merge(bio,
                          left_on='APN_ROW Segment',
                          right_on='APN',
                          how='left',
                          suffixes=('_SS',"_ARCH"))

In [25]:
smartsheets_bio.rename(columns={'APN':'APN_BIO'}, inplace=True)

In [26]:
smartsheets_bio['Bio Monitor Needed_SS'] = smartsheets_bio['Bio Monitor Needed_SS'].fillna("0")
smartsheets_bio['Biological monitoring recommended?'] = smartsheets_bio['Biological monitoring recommended?'].fillna('0')
smartsheets_bio['Observation Date_BIO'] = smartsheets_bio['Observation Date_BIO'].fillna("0")
smartsheets_bio['Bio Assessment_SS'] = smartsheets_bio['Bio Assessment_SS'].fillna("0")

# Smartsheet and ARCH merge and additonal cleaning

In [27]:
smartsheets_ARCH = smartsheets.merge(arch,
                    left_on='APN_ROW Segment',
                    right_on='APN',
                    how='left',
                    suffixes=('_SS',"_ARCH"))

In [28]:
# rename ARCH columns
smartsheets_ARCH.rename(columns={'APN':'APN_ARCH'}, inplace=True)

In [29]:
# FILLNA with 0s
smartsheets_ARCH['Arch Monitor Needed_SS'] = smartsheets_ARCH['Arch Monitor Needed_SS'].fillna("0")
smartsheets_ARCH['Monitoring Recommendation_ARCH'] = smartsheets_ARCH['Monitoring Recommendation_ARCH'].fillna('0')
smartsheets_ARCH['Date_ARCH'] = smartsheets_ARCH['Date_ARCH'].fillna("0")
smartsheets_ARCH['Arch Assessment_SS'] = smartsheets_ARCH['Arch Assessment_SS'].fillna("0")

# Set up column order for each Data base

In [30]:
# ARch and smart sheets
smartsheets_ARCH = smartsheets_ARCH[['APN_ROW Segment', 'APN_ARCH','Street #','Street Name', 'Structural Status','County_SS',
       'Arch Monitor Needed_SS', 'Monitoring Recommendation_ARCH',
                                     'Arch Assessment_SS','Date_ARCH']]

In [31]:
# BIO and smasrt sheets
smartsheets_bio = smartsheets_bio[['APN_ROW Segment', 'APN_BIO','Street #','Street Name', 'Structural Status','County_SS',
                                   'Bio Monitor Needed_SS', 'Biological monitoring recommended?',
                                   'Bio Assessment_SS','Observation Date_BIO']]

# Compare columns in smart sheet to other databases

In [32]:
def compare_cols(df, comp1, comp2):
    if df[comp1] == df[comp2]:
        return 1
    else:
        return 0

# compart ARCH monitor needed to smartsheet

In [33]:
smartsheets_ARCH.insert(smartsheets_ARCH.columns.get_loc('Monitoring Recommendation_ARCH') + 1, "ARC Check",
            smartsheets_ARCH.apply(compare_cols, comp1='Arch Monitor Needed_SS', comp2='Monitoring Recommendation_ARCH',axis=1))

# comapre dates for ARCH and smartsheets dates

In [34]:
smartsheets_ARCH.insert(smartsheets_ARCH.columns.get_loc('Date_ARCH') + 1, 'Check Arch Dates',
            smartsheets_ARCH.apply(compare_cols, comp1='Arch Assessment_SS', comp2='Date_ARCH', axis=1))

# check BIO and Smartsheet dates

In [35]:
smartsheets_bio.insert(smartsheets_bio.columns.get_loc('Observation Date_BIO') + 1, 'Check BIO Dates',
            smartsheets_bio.apply(compare_cols, comp1='Bio Assessment_SS', comp2='Observation Date_BIO',axis=1))

# Compare BIO monitor needed to  smartsheets 

In [36]:
smartsheets_bio.insert(smartsheets_bio.columns.get_loc('Biological monitoring recommended?') + 1, 'BIO Check',
                                  smartsheets_bio.apply(compare_cols, comp1='Bio Monitor Needed_SS',
                                              comp2='Biological monitoring recommended?',axis=1))

# highligh the columns that are comparing values

In [37]:
df_style_BIO = smartsheets_bio.style.set_properties(**{'background-color': '#ffffb3'},
                          subset=['BIO Check', 'Check BIO Dates'])

In [38]:
df_style_ARCH = smartsheets_ARCH.style.set_properties(**{'background-color': '#ffffb3'},
                                                    subset=["ARC Check", 'Check Arch Dates'])

In [39]:
df_style_BIO.to_excel('BIO checks.xlsx')

In [40]:
df_style_ARCH.to_excel('ARCH check.xlsx')